In [3]:
!pip3 install rake-nltk

  Created wheel for rake-nltk: filename=rake_nltk-1.0.4-py2.py3-none-any.whl size=7819 sha256=ea2fb16e3e627f377a03ed85ec7b56699b1f7d17478700327f5ff42b7f010e68
  Stored in directory: /root/.cache/pip/wheels/ef/92/fc/271b3709e71a96ffe934b27818946b795ac6b9b8ff8682483f
Successfully built rake-nltk


In [44]:
!pip3 install mrakun

     |████████████████████████████████| 4.0MB 2.9MB/s 
     |████████████████████████████████| 77.4MB 54kB/s 
     |████████████████████████████████| 235kB 53.8MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
  Created wheel for mrakun: filename=mrakun-0.43-cp36-none-any.whl size=18971 sha256=72f2ce60ddd70f3b0e9851cfdb6700d630acfdcc2bc1a539c5ee179357e78313
  Stored in directory: /root/.cache/pip/wheels/05/90/23/64d11805ec58b8471d4eac7e3415cc40006712d3665c7b3eac
  Created wheel for py3plex: filename=py3plex-0.76-cp36-none-any.whl size=10829352 sha256=f5ab33f3e533e605e8c332fc3be1533583011bc1d8a4b518ec2b52d2d0a543cd
  Stored in directory: /root/.cache/pip/wheels/e7/d9/18/874252b01d594c9db9f78e1eee6476200c0d108b19a5101b04
Successfully built mrakun py3plex


In [6]:
!pip install git+https://github.com/boudinfl/pke.git
!python -m nltk.downloader stopwords
!python -m spacy download en

  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-d02pl6vk
  Running command git clone -q https://github.com/boudinfl/pke.git /tmp/pip-req-build-d02pl6vk
     |████████████████████████████████| 245kB 2.9MB/s 
     |████████████████████████████████| 5.0MB 49.8MB/s 
     |████████████████████████████████| 276kB 53.9MB/s 
  Created wheel for pke: filename=pke-1.8.1-cp36-none-any.whl size=8761404 sha256=0877f5fe1e06cdd3aeb1ea6d2cc293e43348bb230a8ded87ed499e489b940379
  Stored in directory: /tmp/pip-ephem-wheel-cache-hdpj05_5/wheels/8d/24/54/6582e854e9e32dd6c632af6762b3a5d2f6b181c2992e165462
  Created wheel for langcodes: filename=langcodes-2.1.0-cp36-none-any.whl size=5068034 sha256=b903dad0992509c5ed29e65278f5bb230201dbbe3a87e0a82b0dbd23a9ce26f3
  Stored in directory: /root/.cache/pip/wheels/f7/aa/f5/285652c17bb68a5f6d30bdfeebb517d210cabe18268fcb80b5
  Created wheel for marisa-trie: filename=marisa_trie-0.7.5-cp36-cp36m-linux_x86_64.whl size=861393 sha256=bc4f291803bb862

In [26]:
from rake_nltk import Rake
rake = Rake()

In [27]:
from pke.unsupervised import YAKE
from nltk.corpus import stopwords

In [46]:
from mrakun import RakunDetector

In [47]:
def rake_rank(text, num=4):
  rake.extract_keywords_from_text(text)
  #print(rake.get_ranked_phrases_with_scores())
  return sorted(rake.get_ranked_phrases_with_scores(), key = lambda x: x[0], reverse=True)[:num]

In [11]:
dir(rake)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_frequency_dist',
 '_build_ranklist',
 '_build_word_co_occurance_graph',
 '_generate_phrases',
 '_get_phrase_list_from_words',
 'degree',
 'extract_keywords_from_sentences',
 'extract_keywords_from_text',
 'frequency_dist',
 'get_ranked_phrases',
 'get_ranked_phrases_with_scores',
 'get_word_degrees',
 'get_word_frequency_distribution',
 'max_length',
 'metric',
 'min_length',
 'punctuations',
 'rank_list',
 'ranked_phrases',
 'stopwords',
 'to_ignore']

In [31]:
stoplist = stopwords.words('english')

In [51]:
%%time
def rakun_rank(text, num=4):
  hyperparameters = {"distance_threshold":2,
                    "distance_method": "editdistance",
                    "num_keywords" : 10,
                    "pair_diff_length":2,
                    "stopwords" : stoplist,
                    "bigram_count_threshold":2,
                    "num_tokens":[1,2],
        "max_similar" : 3, ## n most similar can show up n times
        "max_occurrence" : 3} ## maximum frequency overall

  keyword_detector = RakunDetector(hyperparameters)
  keywords = keyword_detector.find_keywords(text, input_type = "text")
  return sorted(keywords, key = lambda x: x[1], reverse=True)[:num]
  #keyword_detector.visualize_network()

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 9.54 µs


In [55]:
%%time
extractor = YAKE()
def yake_rank(text):
  # 1. Create YAKE keyword extractor
  # 2. Load document
  extractor.load_document(input=text,
                          language='en',
                          normalization=None)

  # 3. Generate candidate 1-gram and 2-gram keywords
  extractor.candidate_selection(n=3, stoplist=stoplist)

  # 4. Calculate scores for the candidate keywords
  extractor.candidate_weighting(window=2,
                                stoplist=stoplist,
                                use_stems=False)

  # 5. Select 10 highest ranked keywords
  # Remove redundant keywords with similarity above 80%
  key_phrases = extractor.get_n_best(n=4, threshold=0.8)
  return key_phrases

CPU times: user 286 µs, sys: 20 µs, total: 306 µs
Wall time: 312 µs


In [56]:
text = '"too much of an echo when on speaker~phone.", "and w/o speaker phone your cheek pushing against the buttons. thanx, sami c."'
rake_rank(text), yake_rank(text)

([(9.0, 'sami c ."'),
  (4.0, 'cheek pushing'),
  (3.0, 'speaker phone'),
  (1.5, 'speaker')],
 [('speaker', 0.2589678339469337),
  ('phone', 0.2589678339469337),
  ('much', 0.28498490135981175),
  ('echo', 0.28498490135981175)])

In [57]:
text = ["see photos attached. we bought a size up", "none of her nike shoes have ever developed holes, even after 6 months of wearing them. is there any warranty on these?", "what can we do about this?"]

for each in text:
  print(each, '\n')
  %time print("rake : ", rake_rank(each), '\n')
  %time print("yake : ", yake_rank(each), '\n')
  %time print("rakun : ", rakun_rank(each), '\n')
  print('...........................')

see photos attached. we bought a size up 

rake :  [(9.0, 'see photos attached'), (1.0, 'size'), (1.0, 'bought')] 

CPU times: user 965 µs, sys: 0 ns, total: 965 µs
Wall time: 979 µs


27-Sep-20 06:50:23 - Initiated a keyword detector instance.
27-Sep-20 06:50:23 - Number of nodes reduced from 3 to 3


yake :  [('see photos attached', 0.006249869637814998), ('see photos', 0.038712704770480444), ('photos attached', 0.038712704770480444), ('see', 0.1531727145735673)] 

CPU times: user 582 ms, sys: 23.9 ms, total: 606 ms
Wall time: 609 ms
rakun :  [('attached', 0.5), ('photos', 0.0), ('bought', 0.0)] 

CPU times: user 2.97 ms, sys: 1 ms, total: 3.97 ms
Wall time: 4.11 ms
...........................
none of her nike shoes have ever developed holes, even after 6 months of wearing them. is there any warranty on these? 

rake :  [(9.0, 'ever developed holes'), (4.0, 'nike shoes'), (4.0, '6 months'), (1.0, 'wearing')] 

CPU times: user 1.09 ms, sys: 5 µs, total: 1.09 ms
Wall time: 1.1 ms


27-Sep-20 06:50:23 - Initiated a keyword detector instance.
27-Sep-20 06:50:23 - Number of nodes reduced from 6 to 6


yake :  [('see photos attached', 0.0025872531000991623), ('ever developed holes', 0.003845561146517792), ('see photos', 0.02111158115940884), ('photos attached', 0.02111158115940884)] 

CPU times: user 478 ms, sys: 25 ms, total: 503 ms
Wall time: 504 ms
rakun :  [('months', 0.30000000000000004), ('holes', 0.30000000000000004), ('wearing', 0.2), ('developed', 0.2)] 

CPU times: user 4.39 ms, sys: 92 µs, total: 4.48 ms
Wall time: 4.53 ms
...........................
what can we do about this? 

rake :  [] 

CPU times: user 785 µs, sys: 0 ns, total: 785 µs
Wall time: 892 µs


27-Sep-20 06:50:24 - Initiated a keyword detector instance.
27-Sep-20 06:50:24 - Number of nodes reduced from 0 to 0


yake :  [('see photos attached', 0.0010326977494654298), ('ever developed holes', 0.0015511324559800282), ('see photos', 0.011576079587222952), ('photos attached', 0.011576079587222952)] 

CPU times: user 558 ms, sys: 20.7 ms, total: 579 ms
Wall time: 581 ms
rakun :  [] 

CPU times: user 2.56 ms, sys: 18 µs, total: 2.58 ms
Wall time: 2.16 ms
...........................


In [ ]:
Noun phrases and named entities that are fully numeric.
• Named entities that belong to the following categories
are filtered out : DATE, TIME, PERCENT, MONEY,
QUANTITY, ORDINAL, CARDINAL. Refer, Spacy’s
named entity documentation2
for details of the tags.
• Standard stopwords are removed.
• Punctuations are removed except ’-’.

• Common adjectives and reporting verbs are removed
if they occur as the first or last token of a noun
phrase/named entity.
• Determiners are removed from the first token of a noun
phrase/named entity.
• First or last tokens of noun phrases/named entities belonging to following parts of speech: INTJ Interjection,
AUX Auxiliary, CCONJ Coordinating Conjunction,
ADP Adposition, DET Interjection, NUM Numeral,
PART Particle, PRON Pronoun, SCONJ Subordinating
Conjunction, PUNCT Punctutation, SYM Symbol, X
Other, are removed. For a detailed reference of each of
these POS tags please refer Spacy’s documentation3
.
• Starting and ending tokens of a noun phrase/named entity
is removed if they belong to a standard list of english
stopwords.
• Starting and ending tokens of a noun phrase/named entity
is removed if they belong to a standard list of english
functional words.

• Get rid of leading/trailing junk characters.
• Handle dangling/backwards parentheses. We don’t allow
’(’ or ’)’ to appear without the other.
• Handle oddly separated hyphenated words.
• Handle oddly separated apostrophe’d words.
• Normalize whitespace.

In [2]:
!pip3 install pytextrank

In [3]:
import spacy, re
from spacy.matcher import Matcher
import pytextrank

boundary = re.compile('^[0-9]$')
pattern_1 = [
            [{"LOWER": "hello"}, {"IS_ASCII": True}],
            [{"LOWER": "hey"}, {"IS_ASCII": True}],
            [{"LOWER": "hi"}, {"IS_ASCII": True}],
            [{'LOWER': 'good'}, {'ENT_TYPE': 'TIME'}, {"IS_ASCII": True}]
        ]
pattern_2 = [{'ORTH': 'thank'}, {'ORTH': 'you'}] #thank you statements
def custom_seg(doc):
    prev = doc[0].text
    length = len(doc)
    for index, token in enumerate(doc):
        if (
            (token.text == '.' and boundary.match(prev) and index!=(length - 1)) or
               token.text == '>' or (token.text == ' ' and prev == '>')
           ):
            doc[index+1].sent_start = False
        prev = token.text
    return doc


tr = pytextrank.TextRank()

class Spacy_Utilities:
    def __init__(self, disable_ops=[]):
        self.nlp = spacy.load('en_core_web_sm')
        self.nlp.add_pipe(custom_seg, before='parser')
        self.nlp.add_pipe(tr.PipelineComponent, name="textrank", last=True)
        self.matcher = Matcher(self.nlp.vocab)
        self.matcher.add("greet", None, *pattern_1)
        self.matcher.add("not_qc", None, pattern_2)
        #self.nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)
    def greet_pattern_matcher(self):
        return self.matcher
    def process_doc(self, text):
        doc = self.nlp(text)
        return doc
    def sent_tokenize(self, text):
        return text.sents
    def ent_words_extract(self, text):
        return [X.text.split() for X in text.ents]
    def ent_labels_extract(self, text):
        return [X.label_ for X in text.ents]


In [6]:
spacy_en = Spacy_Utilities()

In [14]:
 questions = ["The ranked keywords are evaluated using exact match",
"evaluation metric as used in SemEval 2010 Task 5. We match",
"the keywords in the annotated documents in the benchmark. datasets with those generated by our method, and calculate. micro-averaged precision, recall and F-score (β = 1), respectively. In the evaluation, we check the performance over",
"the top 5, 10 and 15 candidates returned by our system."]

In [26]:
tmp_questions_arr = []
for question_doc in spacy_en.nlp.pipe([questions[2]]):
        tmp_arr = [sent.text for sent in question_doc.sents]
        tmp_questions_arr.append( " ".join(tmp_arr[-3:]) )
print(tmp_questions_arr)

['datasets with those generated by our method, and calculate. micro-averaged precision, recall and F-score (β = 1), respectively. In the evaluation, we check the performance over']


In [27]:
import numpy as np
NEUTRAL = 'neutral'
S_POS   = 'slight_positive'
POS     = 'positive'
S_NEG   = 'slight_negative'
NEG     = 'negative'

def decide_label(rslt_arr):
    max_id = np.argmax(rslt_arr)
    if max_id == 1:
        return NEUTRAL
    elif max_id == 0:
        if rslt_arr[0] > 0.6:
            return NEG
        elif rslt_arr[0] <= 0.6 :
            return S_NEG
    elif max_id == 2:
        if rslt_arr[2] > 0.6:
            return POS
        elif rslt_arr[2] <= 0.6:
            return S_POS

In [28]:
import json, requests
from typing import List
def do_sentiment_evaluation(ip_list: List[str]):
    url = f"http://34.72.3.36/v1/models/sentiment:predict"
    headers = {
    'content-type': 'application/json'
    }
    payload = json.dumps({
        "inputs": ip_list
        })
    response = requests.request("POST", url, headers=headers, data = payload)
    rslt = json.loads(response.text.encode('utf8'))
    print(f"SENTIMENT RESULT {rslt}")
    sentiment_op = [decide_label(each) for each in rslt['outputs']]
    return sentiment_op

In [29]:
do_sentiment_evaluation(tmp_questions_arr)

SENTIMENT RESULT {'outputs': [[0.361043811, 0.436594546, 0.202361614]]}


['neutral']